# OpenVINOを使ったC++プロジェクトを作る方法

ここでは、OpenVINOを使った簡単な画像識別(classification)アプリケーションをC++で作成する方法、実行する方法について学びます。
- OpenVINO C++プロジェクトの作り方、ビルド方法 (CMake)

#### Windowsでビルドする場合
OpenVINOパッケージに加えていくつかのソフトウエアをインストールする必要があります。  
- Visual Studio (2019, C++)
- CMake (x64用)
- Python 3.7+ (x64)

また、Visual Studioのツール類へのパスを通すため、スタートメニューから「Visual Studio 2019 / x64 Native Tools Command Prompt」でコマンドプロンプトを開き、そこからJupyterを起動してください。通常のコマンドプロンプト(cmd.exe)からJupyterを開くとパスが通っていないためビルドに失敗します。  
ビルドにはIntel oneAPI base tool kitのDPC++ compiler (Data Parallel C++)も使用可能ですがここではその手順の紹介はしていません。

----
## 1. C++プログラムをビルドし、実行する

### 1.1. DLモデルの準備
OpenVINOを使った推論プログラムで利用するIRモデル(DLモデル)を準備します。今回のプログラムは画像の分類(classification)ですので、それに合うモデルをダウンロードします。ここでは`squeezenet1.1`モデルを使用します。  
ここではモデルのダウンロードにはOpenVINO付属の`Model downloder`を、モデルをIRモデルに変換するのには`Model converter`を使用しています。実行ログの最後のほうに`[ SUCCESS ]`  と5つ表示されればIRモデルへの変換が完了です。変換されたIRモデルは`./public/squeezenet1.1/FP16/`の中に格納されています。

**Memo:** 通常、TensorFlow, ONNX, CaffeなどのモデルデータをIRモデルデータに変換するには`Model optimizer`を使用します。`Model converter`は便利なツールですが、`Model downloader`でダウンロードしてきたモデルにしか使えません。

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name squeezenet1.1 --precisions FP16

In [ ]:
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\downloader.py" --name squeezenet1.1
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\converter.py"  --name squeezenet1.1 --precisions FP16

### 1.2. 推論に使用する入力画像とクラスラベルデータを準備する
- 入力画像
 -- OpenVINOのデモプログラム実行テスト用の画像ファイルをコピーしてきます(`car.png`)
- クラスラベルデータ
 -- 今回の`squeezenet1.1`モデルはImageNetのデータセットで学習されたモデルですので、画像を1000のクラスに分類します
 -- プログラムでクラス番号だけを表示しても意味が分からないので、ラベルテキストデータ(`synset_words.txt`)をOpenVINOサンプルディレクトリからコピーしクラス名を表示できるようにします

In [ ]:
# 入力画像データをコピー
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
# クラスラベルテキストデータをコピー
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

from IPython.display import Image
Image('car.png')

In [ ]:
# Windows - 入力画像データをコピー
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car.png" .
# クラスラベルテキストデータをコピー
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\squeezenet1.1.labels" synset_words.txt

from IPython.display import Image
Image('car.png')

### 1.3. C++ソースコードを用意する
ここではワークショップの便宜上`%%writefile`マジックコマンドでソースコードファイルを生成していますが、テキストエディタなど他の手段でも問題ありません(むしろそちらの方が普通でしょう)。ここでは`main.cpp`という名前でソースファイルを用意しています。

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <fstream>
#include <vector>

#include <opencv2/opencv.hpp>
#include <inference_engine.hpp>

namespace ie = InferenceEngine;

int main(int argc, char *argv[]) {

    std::ifstream label_file("synset_words.txt");  
    std::string str;
    std::vector<std::string> labels;
    while(getline(label_file, str)) labels.push_back(str);
    label_file.close();

    // Creating an Inference Engine core object
    ie::Core ie;

    // Loading a DL model to memory
    ie::CNNNetwork network = ie.ReadNetwork("public/squeezenet1.1/FP16/squeezenet1.1.xml",
                                            "public/squeezenet1.1/FP16/squeezenet1.1.bin");
    // Setting up the input blob
    std::shared_ptr<ie::InputInfo> input_info = network.getInputsInfo().begin()->second;
    std::string input_name = network.getInputsInfo().begin()->first;
    input_info->getPreProcess().setResizeAlgorithm(ie::RESIZE_BILINEAR);
    input_info->setLayout(ie::Layout::NHWC);
    input_info->setPrecision(ie::Precision::U8);

    // Setting up the output blob
    ie::DataPtr output_info = network.getOutputsInfo().begin()->second;
    std::string output_name = network.getOutputsInfo().begin()->first;
    output_info->setPrecision(ie::Precision::FP32);

    // Set the DL model to an Inference Engine object
    ie::ExecutableNetwork executable_network = ie.LoadNetwork(network, "CPU");
    ie::InferRequest infer_request = executable_network.CreateInferRequest();

// main inferencing loop - start -------

    cv::Mat image = cv::imread("car.png");   // Loading an image

    ie::TensorDesc tDesc(ie::Precision::U8, 
        {1, 3, static_cast<long unsigned int>(image.rows), 
               static_cast<long unsigned int>(image.cols) }, ie::Layout::NHWC);
    infer_request.SetBlob(input_name, ie::make_shared_blob<uint8_t>(tDesc, image.data));

    infer_request.Infer();                   // Do inferencing

    // Displaying result
    float* output = infer_request.GetBlob(output_name)->buffer();
    std::cout << "\nresults\n------------------" << std::endl;
    std::vector<int> idx;
    for(int i=0; i<1000; i++) idx.push_back(i);
    std::sort(idx.begin(), idx.end(), [output](const int& left, const int& right) { return output[left]>output[right]; } );
    for (size_t id = 0; id < 5; ++id)  std::cout << id <<  " : " << idx[id] << " : " << 
        std::fixed<< std::setprecision(2) << output[idx[id]]*100 << "% " << labels[idx[id]] << std::endl;

// main inferencing loop - end ------
}

### 1.4. `CMakeLists.txt`を用意する
ここでは`cmake`を使ってC++ソースのビルドを行いますので、そのための設定ファイル(`CMakeLists.txt`)を`%%writefile`コマンドで用意しています。  
`set(TARGET_NAME`の部分と`add_executable(`の部分を書き換えれば、別のプロジェクトに流用できます。もちろん、追加のライブラリなどを利用する場合はそれらを`target_link_libraries(`に追加したり、`find_package(`を追加したりする必要があります。

In [ ]:
%%writefile CMakeLists.txt
# Sample CMakeLists.txt file for an OpenVINO Inference Engine project
project(Project)
cmake_minimum_required (VERSION 2.8.1)

set(TARGET_NAME simple_cnn)     # name of executable file
set(CMAKE_BUILD_TYPE "Release")

set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUIRED ON)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -std=c++11 -fPIE")

find_package(InferenceEngine 1.1 REQUIRED)
find_package(OpenCV REQUIRED)
add_definitions(-DUSE_OPENCV)

include_directories( ${InferenceEngine_INCLUDE_DIRS} ${OpenCV_INCLUDE_DIRS} )
link_directories( )

add_executable( ${TARGET_NAME} main.cpp )    # list of source file(s)
set_target_properties(${TARGET_NAME} PROPERTIES "CMAKE_CXX_FLAGS" "${CMAKE_CXX_FLAGS}")
target_link_libraries(${TARGET_NAME} ${InferenceEngine_LIBRARIES} ${OpenCV_LIBS} )


### 1.5. ビルドを行う
`build`ディレクトリを作成しC++ソースをビルドします。  
`!`コマンドは1行ごとにシェルを抜けてしまいますので、`cd`コマンドなどシェルの状態を変えるコマンドを発行しても現在の行の実行が終わるとともに状態が破棄されてしまいます(コマンドは子シェルで実行されます)。そのため、`&&`でコマンドをつなぎ、１行でビルドしています。  
ビルドされた実行可能バイナリは`./build/simple_cnn` または`build\Release\simple_cnn.exe`です。
Windowsの場合はVisual Studio 2019を使ってビルドすることを想定しています。ここではmsbuildコマンドを使ってコマンドラインビルドを行っていますが、cmakeが生成するProject.slnファイルをVisual Studioで開き、IDE上でビルドすることも可能です。

In [ ]:
# Linux
!mkdir -p build && cd build && cmake .. && make && ls -l

In [ ]:
# Windows
!if not exist build (mkdir build)
!cd build && cmake -G "Visual Studio 16 2019" .. && msbuild Project.sln /t:clean;rebuild /p:Configuration=Release;Platform="x64" && cd ..

### 1.6. ビルドしたプログラムを実行する
ビルドしたexecutableを実行してみます。  
このプログラムは引数を取りません。使用するファイル名(モデル名、入力画像ファイル名、クラスラベルファイル名)はプログラム中にハードコードされています。  
ただしく推論結果が表示されることを確認してください。

In [ ]:
# Linux
!build/simple_cnn

In [ ]:
# Windows
!build\Release\simple_cnn.exe

----
ここまででOpenVINOを使った簡単なC++プログラムをDevCloud上でビルド、実行する方法を学びました。  
今回のプログラムを小改造していろいろ試したり、実行したりしてみてください。

<おわり>